# Generating IBOR extract

## Setup

In [1]:
import os
import json
from datetime import datetime, timedelta
import pytz

import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

import pandas as pd

pd.set_option('display.max_columns', None)

secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name='LusidJupyterNotebook'
)

In [2]:
with open(r'config/build_transactions_mapping.json') as mapping_file:
    build_txn_json_mappings = json.load(mapping_file)

## Load transactions into a new scope

In [3]:
scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [4]:
transactions_file = r'data/equity_transactions.csv'
transactions_df = pd.read_csv(transactions_file)
transactions_df['portfolio_code'] = portfolio_code

In [5]:
seed_data_response = seed_data(api_factory,
                              ['portfolios', 'instruments', 'transactions'],
                              scope,
                              transactions_file,
                              'csv')

## Get Holdings

In [6]:
txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [7]:
holdings_response = txn_port_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name']
)

In [8]:
lusid_response_to_data_frame(
    holdings_response,
    rename_properties=True
).head(10)

,instrument_scope,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
0,default,LUID_00003D60,{},Aviva,EQUITY_UK,3a48-faea-3ca5-56,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_00003D61,{},BHP,EQUITY_UK,3a48-faea-3ca5-56,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_00003D68,{},Barclays,EQUITY_UK,3a48-faea-3ca5-56,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_00003D67,{},BP,EQUITY_UK,3a48-faea-3ca5-56,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_00003D62,{},HSBC,EQUITY_UK,3a48-faea-3ca5-56,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},GBP,EQUITY_UK,3a48-faea-3ca5-56,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_00003D63,{},Morrisons,EQUITY_UK,3a48-faea-3ca5-56,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_00003D5Y,{},Tesco,EQUITY_UK,3a48-faea-3ca5-56,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_00003D65,{},Rightmove,EQUITY_UK,3a48-faea-3ca5-56,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_00003D5Z,{},vodafone,EQUITY_UK,3a48-faea-3ca5-56,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP


## Holdings for a specific date and time

In [9]:
holdings_response_fifth_jan = txn_port_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-05T12:30:00Z",
    property_keys=["Instrument/default/Name"]
)

In [16]:
holdings_fifth_jan = lusid_response_to_data_frame(holdings_response_fifth_jan, rename_properties=True)

column_rename = {
    "instrument_uid": "luid",
    "Name(default-Properties)": "instrumentName",
    "holding_type": "holdingType",
    "units": "units",
    "settled_units": "settledUnits",
    "cost.amount": "costAmount",
    "cost.currency": "costCurrency",
    "cost_portfolio_ccy.currency": "portfolioCurrency"
}

holdings_fifth_jan = holdings_fifth_jan.rename(columns = column_rename)[column_rename.values()].copy()
display(holdings_fifth_jan.query("instrumentName == 'Barclays'"))

,luid,instrumentName,holdingType,units,settledUnits,costAmount,costCurrency,portfolioCurrency
2,LUID_00003D68,Barclays,P,300000.0,300000.0,600000.0,GBP,GBP


## What transactions produced my holdings?

In [19]:
build_transactions_response = txn_port_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    transaction_query_parameters = models.TransactionQueryParameters(start_date="2020-01-01",
                                                                                   end_date="2020-12-31")
)

In [23]:
columns = ["TransactionId",
"TransactionType",
"LusidInstrumentId",
"InstrumentName",
"SettlementDate",
"Price",
"Units",
"ResultantHolding"]

transactions_df = lusid_response_to_data_frame(build_transactions_response, 
                                               column_name_mapping=build_txn_json_mappings,
                                                rename_properties=True)
transactions_df = transactions_df[columns]
transactions_df

,TransactionId,TransactionType,LusidInstrumentId,InstrumentName,SettlementDate,Price,Units,ResultantHolding
0,trd_0001,Buy,LUID_00003D60,Aviva,2020-01-04 00:00:00+00:00,5.0,120000.0,120000.0
1,trd_0002,Buy,LUID_00003D60,Aviva,2020-01-04 00:00:00+00:00,5.0,12000.0,132000.0
2,trd_0003,Buy,LUID_00003D61,BHP,2020-01-04 00:00:00+00:00,18.0,60000.0,60000.0
3,trd_0004,Buy,LUID_00003D61,BHP,2020-01-04 00:00:00+00:00,18.0,60000.0,120000.0
4,trd_0005,Buy,LUID_00003D68,Barclays,2020-01-04 00:00:00+00:00,2.0,150000.0,150000.0
5,trd_0006,Buy,LUID_00003D68,Barclays,2020-01-04 00:00:00+00:00,2.0,150000.0,300000.0
6,trd_0007,Buy,LUID_00003D67,BP,2020-01-04 00:00:00+00:00,5.0,100000.0,100000.0
7,trd_0008,Buy,LUID_00003D67,BP,2020-01-04 00:00:00+00:00,5.0,100000.0,200000.0
8,trd_0009,Buy,LUID_00003D62,HSBC,2020-01-04 00:00:00+00:00,6.0,20000.0,20000.0
9,trd_0010,Buy,LUID_00003D62,HSBC,2020-01-04 00:00:00+00:00,6.0,20000.0,40000.0


In [25]:
holdings_fifth_jan.to_excel('extracts/holdings_20200105.xlsx')